In [5]:
# import libraries and the CIFAR-100 dataset

# import the CIFAR-100 dataset
from keras.datasets import cifar100
import tensorflow as tf
# import the libraries
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.applications import VGG16,MobileNet
from keras.applications import ResNet50
from keras.models import Model
import numpy as np
import pandas as pd
from keras.src.callbacks import EarlyStopping
from keras.src.layers import Dropout
from sklearn.metrics import recall_score, precision_score, f1_score

In [16]:
# load the CIFAR-100 dataset

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255

In [17]:
# Conversion of labels into one-hot encoding categories

# initialize the number of classes
number_classes = 100

y_train = to_categorical(y_train, num_classes=number_classes)
y_test = to_categorical(y_test, num_classes=number_classes)

In [18]:
# Separation of the training set into 60%, 40%, 20%

x_train_60, _, y_train_60, _ = train_test_split(x_train, y_train, train_size=0.6,
                                                random_state=101)
x_train_40, _, y_train_40, _ = train_test_split(x_train, y_train, train_size=0.4,
                                                random_state=101)
x_train_20, _, y_train_20, _ = train_test_split(x_train, y_train, train_size=0.2,
                                                random_state=101)

# Check the sizes
print("60% Training Set:", x_train_60.shape, y_train_60.shape)
print("40% Training Set:", x_train_40.shape, y_train_40.shape)
print("20% Training Set:", x_train_20.shape, y_train_20.shape)

60% Training Set: (30000, 32, 32, 3) (30000, 100)
40% Training Set: (20000, 32, 32, 3) (20000, 100)
20% Training Set: (10000, 32, 32, 3) (10000, 100)


In [9]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [ ]:
# using the 60% of the dataset

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D
from keras.callbacks import ReduceLROnPlateau

# Load ResNet50 with pre-trained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
]

datagen.fit(x_train_60)
history = model.fit(datagen.flow(x_train_60, y_train_60, batch_size=32), epochs=50, validation_data=(x_test, y_test), callbacks=callbacks)
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 209s 215ms/step - accuracy: 0.0889 - loss: 4.5379 - val_accuracy: 0.2127 - val_loss: 3.2199 - learning_rate: 0.0010
Epoch 2/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 195s 208ms/step - accuracy: 0.1752 - loss: 3.5181 - val_accuracy: 0.2449 - val_loss: 3.0421 - learning_rate: 0.0010
Epoch 3/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 174s 185ms/step - accuracy: 0.1859 - loss: 3.4112 - val_accuracy: 0.2646 - val_loss: 2.9562 - learning_rate: 0.0010
Epoch 4/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 188s 200ms/step - accuracy: 0.2039 - loss: 3.3101 - val_accuracy: 0.2701 - val_loss: 2.9575 - learning_rate: 0.0010
Epoch 5/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 230s 245ms/step - accuracy: 0.2068 - loss: 3.2723 - val_accuracy: 0.2695 - val_loss: 2.9433 - learning_rate: 0.0010
Epoch 6/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 219s 234ms/step - accuracy: 0.2130 - loss: 3.2442 - val_accuracy: 0.2778 - val_loss: 2.9408 - learning_rate: 0.0010
Epoch 7/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 222s 237ms/step - accura

In [15]:
# Predict classes and probabilities on the test set
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_60 = accuracy_score(y_true, y_pred)

# Calculate precision, recall, and F1-score
precision_60 = precision_score(y_true, y_pred, average='macro')
recall_60 = recall_score(y_true, y_pred, average='macro')
f1_60 = f1_score(y_true, y_pred, average='macro')

# Print the results
print(f"Accuracy: {accuracy_60:.4f}")
print(f"Precision: {precision_60:.4f}")
print(f"Recall: {recall_60:.4f}")
print(f"F1-Score: {f1_60:.4f}")

938/938 ━━━━━━━━━━━━━━━━━━━━ 129s 138ms/step
Accuracy: 0.4684
Precision: 0.4973
Recall: 0.4678
F1-Score: 0.4651


In [12]:
# using the 40% of the dataset

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load ResNet50 with pre-trained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_40)
history = model.fit(datagen.flow(x_train_40, y_train_40, batch_size=128), epochs=10, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/10


C:\Users\nick-\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 162s 245ms/step - accuracy: 0.1448 - loss: 4.1166 - val_accuracy: 0.2470 - val_loss: 3.0494
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 154s 246ms/step - accuracy: 0.2865 - loss: 2.8677 - val_accuracy: 0.2848 - val_loss: 2.8586
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 213ms/step - accuracy: 0.3266 - loss: 2.6478 - val_accuracy: 0.2971 - val_loss: 2.8284
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 124s 199ms/step - accuracy: 0.3577 - loss: 2.4955 - val_accuracy: 0.3039 - val_loss: 2.8388
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 150s 240ms/step - accuracy: 0.3766 - loss: 2.3943 - val_accuracy: 0.3071 - val_loss: 2.8686
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 135s 216ms/step - accuracy: 0.4014 - loss: 2.2959 - val_accuracy: 0.3069 - val_loss: 2.8948
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 227ms/step - accuracy: 0.4091 - loss: 2.2336 - val_accuracy: 0.3086 - val_loss: 2.9014
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 127s 203ms/step - accuracy: 0.4310 - loss: 2.16

In [14]:
# Predict classes and probabilities on the test set
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_40 = accuracy_score(y_true, y_pred)

# Calculate precision, recall, and F1-score
precision_40 = precision_score(y_true, y_pred, average='macro')
recall_40 = recall_score(y_true, y_pred, average='macro')
f1_40 = f1_score(y_true, y_pred, average='macro')

# Print the results
print(f"Accuracy: {accuracy_40:.4f}")
print(f"Precision: {precision_40:.4f}")
print(f"Recall: {recall_40:.4f}")
print(f"F1-Score: {f1_40:.4f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 95s 147ms/step
Accuracy: 0.4663
Precision: 0.4933
Recall: 0.4652
F1-Score: 0.4617


In [10]:
# using the 20% of the dataset

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load ResNet50 with pre-trained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

# Adding custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_20)
history = model.fit(datagen.flow(x_train_20, y_train_20, batch_size=128), epochs=10, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 262ms/step - accuracy: 0.1161 - loss: 4.6378 - val_accuracy: 0.2126 - val_loss: 3.3009
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 84s 268ms/step - accuracy: 0.2758 - loss: 2.9597 - val_accuracy: 0.2465 - val_loss: 3.1457
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 96s 305ms/step - accuracy: 0.3333 - loss: 2.6074 - val_accuracy: 0.2645 - val_loss: 3.0461
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 88s 281ms/step - accuracy: 0.3676 - loss: 2.4486 - val_accuracy: 0.2689 - val_loss: 3.0606
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 79s 251ms/step - accuracy: 0.4026 - loss: 2.2632 - val_accuracy: 0.2718 - val_loss: 3.1121
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 93s 297ms/step - accuracy: 0.4370 - loss: 2.1313 - val_accuracy: 0.2812 - val_loss: 3.1506
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 78s 251ms/step - accuracy: 0.4558 - loss: 2.0326 - val_accuracy: 0.2879 - val_loss: 3.2042
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 88s 281ms/step - accuracy: 0.4896 - loss: 1

In [11]:
# Predict classes and probabilities on the test set
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_20 = accuracy_score(y_true, y_pred)

# Calculate precision, recall, and F1-score
precision_20 = precision_score(y_true, y_pred, average='macro')
recall_20 = recall_score(y_true, y_pred, average='macro')
f1_20 = f1_score(y_true, y_pred, average='macro')

# Print the results
print(f"Accuracy: {accuracy_20:.4f}")
print(f"Precision: {precision_20:.4f}")
print(f"Recall: {recall_20:.4f}")
print(f"F1-Score: {f1_20:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 46s 140ms/step
Accuracy: 0.6103
Precision: 0.6311
Recall: 0.6082
F1-Score: 0.6068


In [ ]:
# using the 60% of the dataset
# using the second model, VGG16

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load VGG16 with pre-trained ImageNet weights
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model_vgg.trainable = False

# Adding custom layers
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model_vgg.input, outputs=predictions)

for layer in base_model_vgg.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_60)
history = model.fit(datagen.flow(x_train_60, y_train_60, batch_size=128), epochs=10, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
# Predict classes and probabilities on the test set
y_pred_probs_vgg = model.predict(x_test)
y_pred_vgg = np.argmax(y_pred_probs_vgg, axis=1)
y_true_vgg = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_60_vgg = accuracy_score(y_true_vgg, y_pred_vgg)

# Calculate precision, recall, and F1-score
precision_60_vgg = precision_score(y_true_vgg, y_pred_vgg, average='macro')
recall_60_vgg = recall_score(y_true_vgg, y_pred_vgg, average='macro')
f1_60_vgg = f1_score(y_true_vgg, y_pred_vgg, average='macro')

# Print the results
print(f"Accuracy: {accuracy_60_vgg:.4f}")
print(f"Precision: {precision_60_vgg:.4f}")
print(f"Recall: {recall_60_vgg:.4f}")
print(f"F1-Score: {f1_60_vgg:.4f}")

In [ ]:
# using the 40% of the dataset
# using the second model, VGG16

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load VGG16 with pre-trained ImageNet weights
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model_vgg.trainable = False

# Adding custom layers
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model_vgg.input, outputs=predictions)

for layer in base_model_vgg.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_40)
history = model.fit(datagen.flow(x_train_40, y_train_40, batch_size=128), epochs=10, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
# Predict classes and probabilities on the test set
y_pred_probs_vgg = model.predict(x_test)
y_pred_vgg = np.argmax(y_pred_probs_vgg, axis=1)
y_true_vgg = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_40_vgg = accuracy_score(y_true_vgg, y_pred_vgg)

# Calculate precision, recall, and F1-score
precision_40_vgg = precision_score(y_true_vgg, y_pred_vgg, average='macro')
recall_40_vgg = recall_score(y_true_vgg, y_pred_vgg, average='macro')
f1_40_vgg = f1_score(y_true_vgg, y_pred_vgg, average='macro')

# Print the results
print(f"Accuracy: {accuracy_40_vgg:.4f}")
print(f"Precision: {precision_40_vgg:.4f}")
print(f"Recall: {recall_40_vgg:.4f}")
print(f"F1-Score: {f1_40_vgg:.4f}")

In [ ]:
# using the 20% of the dataset
# using the second model, VGG16

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load VGG16 with pre-trained ImageNet weights
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model_vgg.trainable = False

# Adding custom layers
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model_vgg.input, outputs=predictions)

for layer in base_model_vgg.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_20)
history = model.fit(datagen.flow(x_train_20, y_train_20, batch_size=32), epochs=50, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
# Predict classes and probabilities on the test set
y_pred_probs_vgg = model.predict(x_test)
y_pred_vgg = np.argmax(y_pred_probs_vgg, axis=1)
y_true_vgg = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_20_vgg = accuracy_score(y_true_vgg, y_pred_vgg)

# Calculate precision, recall, and F1-score
precision_20_vgg = precision_score(y_true_vgg, y_pred_vgg, average='macro')
recall_20_vgg = recall_score(y_true_vgg, y_pred_vgg, average='macro')
f1_20_vgg = f1_score(y_true_vgg, y_pred_vgg, average='macro')

# Print the results
print(f"Accuracy: {accuracy_20_vgg:.4f}")
print(f"Precision: {precision_20_vgg:.4f}")
print(f"Recall: {recall_20_vgg:.4f}")
print(f"F1-Score: {f1_20_vgg:.4f}")

In [ ]:
import pandas as pd

#create dataframe to store the results into a table

# initialize the data-results

data = {
    'Metric': ['accuracy', 'recall', 'precision', 'F1-score'],
    'ResNet50 60% training set': [accuracy_60, recall_60, precision_60, f1_60],
    'VGG16 60% training set': [accuracy_60_vgg, recall_60_vgg, precision_60_vgg, f1_60_vgg],
    'ResNet50 40% training set': [accuracy_40, recall_40, precision_40, f1_40],
    'VGG16 40% training set': [accuracy_40_vgg, recall_40_vgg, precision_40_vgg, f1_40_vgg],
    'ResNet50 20% training set': [accuracy_20, recall_20, precision_20, f1_20],
    'VGG16 20% training set': [accuracy_20_vgg, recall_20_vgg, precision_20_vgg, f1_20_vgg]
}

# dataframe creation
df = pd.DataFrame(data)

df

In [15]:
# using the 20% of the dataset
# using the second model, VGG16

from sklearn.metrics import accuracy_score
from keras.src.layers import GlobalAveragePooling2D

# Load VGG16 with pre-trained ImageNet weights
base_model_vgg = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model_vgg.trainable = False

# Adding custom layers
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(100, activation='softmax')(x)
model = Model(inputs=base_model_vgg.input, outputs=predictions)

for layer in base_model_vgg.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

datagen.fit(x_train_20)
history = model.fit(datagen.flow(x_train_20, y_train_20, batch_size=32), epochs=50, validation_data=(x_test, y_test))
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 109s 310ms/step - accuracy: 0.0081 - loss: 4.8451 - val_accuracy: 0.0100 - val_loss: 4.6053
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.0108 - loss: 4.6044

KeyboardInterrupt: 

In [ ]:
# Predict classes and probabilities on the test set
y_pred_probs_vgg = model.predict(x_test)
y_pred_vgg = np.argmax(y_pred_probs_vgg, axis=1)
y_true_vgg = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy_20_vgg = accuracy_score(y_true_vgg, y_pred_vgg)

# Calculate precision, recall, and F1-score
precision_20_vgg = precision_score(y_true_vgg, y_pred_vgg, average='macro')
recall_20_vgg = recall_score(y_true_vgg, y_pred_vgg, average='macro')
f1_20_vgg = f1_score(y_true_vgg, y_pred_vgg, average='macro')

# Print the results
print(f"Accuracy: {accuracy_20_vgg:.4f}")
print(f"Precision: {precision_20_vgg:.4f}")
print(f"Recall: {recall_20_vgg:.4f}")
print(f"F1-Score: {f1_20_vgg:.4f}")